# Exploring performance

In this demo, we ...

In [ ]:
## Installing (if not) and importing compiam to the project
import importlib.util
if importlib.util.find_spec('compiam') is None:
    %pip install compiam
if importlib.util.find_spec('essentia') is None:
    %pip install essentia
if importlib.util.find_spec('torch') is None:
    %pip install "torch==1.13"
if importlib.util.find_spec('tensorflow') is None:
    %pip install "tensorflow==2.15.0" "keras<3"

import compiam
import essentia.standard as estd

# Import extras and supress warnings to keep the tutorial clean
import os
import gdown
import zipfile

import numpy as np
import IPython.display as ipd

from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

AUDIO_PATH = os.path.join("..", "audio", "demos")
ARTIST = "dr-brindha-manickavasakan"

We will work on a concert led by Dr. Brindha Manickavasakan, a well-known Carnatic music performer and Doctor which has been very much involved in our research efforts. This concert happened within the December Season 2023 in Chennai, India, at the well-known Arkay Convention Centre. Please note this recording in particular is part of the newly published Saraga Audiovisual Dataset (A. Shankar et al., 2024), which will be soon available for access through ``mirdata`` and ``compIAM``.

For now, we will download this particular concert and explore a given rendition.

In [ ]:
url = "https://drive.google.com/uc?id=1iR0bfxDLQbH8fEeHU_GFsg2kh7brZ0HZ&export=download"
output =  os.path.join(AUDIO_PATH,  "dr-brindha-manickavasakan.zip")
gdown.download(url, output, quiet=False) 

Once the audio is download, we can extract all the files and remove the .zip file.

In [ ]:
# Unzip file
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(AUDIO_PATH)

# Delete zip file after extraction
os.remove(output)

# Loading and visualising the data

In [ ]:
rendition = "Bhavanuta"  # Selecting a rendition

We use ``Essentia`` to first load the mixture audio of the concert. The function ``AudioLoader`` can be used to load an audio signal from a file path and return the signal plus some important technical information about it.

In [ ]:
file_path = os.path.join(AUDIO_PATH, ARTIST, rendition + ".wav")
audio_mix, _, _, _, _, _ = estd.AudioLoader(filename=file_path)()
audio_mix = audio_mix.T  # Put channels first

Let's quickly listed to 30 seconds of this incredible performance!

In [ ]:
ipd.Audio(audio_mix[..., :44100*30], rate=44100)

# Low level feature extraction

## Tonic Identification

## Music Source Separation

Small intro goes here?

In [ ]:
from compiam import load_model
# This model uses tensorflow in the backend!
separation_model = load_model("separation:cold-diff-sep")

In [ ]:
separated_vocals = separation_model.separate(audio_mix)
separated_vocals.shape

In [ ]:
ipd.Audio(separated_vocals[..., :44100*30], rate=44100)

For further reference, please visit the [music source separation page](singing-voice-extraction).

## Pitch Extraction


In [ ]:
from compiam import load_model
# This model uses tensorflow in the backend!
ftanet_carnatic = load_model("melody:ftanet-carnatic")

In [ ]:
separated_vocals = separation_model.separate(audio_mix)

For further reference, please visit the [pitch extraction page](melody-extraction).

## Percussion onset detection

# High level feature extraction

## Melodic pattern discovery

## Raga recognition

In [ ]:
from compiam import load_model
# This model uses tensorflow in the backend!
deepsrgm = load_model("melody:deepsrgm")

In [ ]:
feat = deepsrgm.get_features(file_path)

In [ ]:
predicted_raga = deepsrgm.predict(feat)
predicted_raga

In [ ]:
deepsrgm.raga_mapping[predicted_raga]